## Prepare Snowpark Session

Create a Snowpark Session using your Snowflake account credentials. For more information about creating a
`Session`, see [Creating a Session for Snowpark Python](https://docs.snowflake.com/en/developer-guide/snowpark/python/creating-session).

In [1]:
from snowflake.snowpark import Session, context, exceptions

try:
    # Retrieve active session if in Snowpark Notebook
    session = context.get_active_session()
except exceptions.SnowparkSessionException:
    # ACTION REQUIRED: Need to manually configure Snowflake connection if using Jupyter
    connection_parameters = {
        "account": "<your snowflake account>",
        "user": "<your snowflake user>",
        "password": "<your snowflake password>",
    }

    session = Session.builder.configs(connection_parameters).create()

Prepare a database and schema to use for this example. We recommend creating a new schema for easy cleanup.

In [2]:
# ACTION REQUIRED: Set these values based on your environment.
# We recommend creating a new schema for this example.
DATABASE_NAME = "<database name>"
SCHEMA_NAME = "<schema name>"
WAREHOUSE_NAME = "<warehouse name>"

# Uncomment the line below to create a new database
# session.sql(f"CREATE DATABASE IF NOT EXISTS {DATABASE_NAME}").collect()

# Create a new schema and select a warehouse to use
session.sql(f"CREATE SCHEMA {DATABASE_NAME}.{SCHEMA_NAME}").collect()
session.use_warehouse(WAREHOUSE_NAME)

## Prepare sample data

For this exercise, we will use the Citi Bike NYC bike share dataset from the
[Zero to Snowflake tutorial](https://developers.snowflake.com/solution/citi-bike-data-analysis-create-and-manage-snowflake-objects-using-notebooks/).
The data is hosted in an Amazon AWS S3 bucket as CSV files; we'll mount the data as an
[External Stage](https://docs.snowflake.com/en/user-guide/data-load-s3-create-stage)
and load the data into a temporary table.

In [3]:
from snowflake.snowpark.types import (
    StructType,
    StructField,
    IntegerType,
    FloatType,
    StringType,
    TimestampType,
)

# Create an external stage that points to the AWS S3 bucket.
STAGE_NAME = f"{DATABASE_NAME}.{SCHEMA_NAME}.citibike_trips"
S3_BUCKET_URL = "s3://snowflake-workshop-lab/citibike-trips-csv/"
session.sql(f"CREATE STAGE IF NOT EXISTS {STAGE_NAME} URL = '{S3_BUCKET_URL}'").collect()

# Create a DataFrame that loads the data from the stage
schema = StructType([
    StructField("tripduration", IntegerType()),
    StructField("starttime", TimestampType()),
    StructField("stoptime", TimestampType()),
    StructField("start_station_id", IntegerType()),
    StructField("start_station_name", StringType()),
    StructField("start_station_latitude", FloatType()),
    StructField("start_station_longitude", FloatType()),
    StructField("end_station_id", IntegerType()),
    StructField("end_station_name", StringType()),
    StructField("end_station_latitude", FloatType()),
    StructField("end_station_longitude", FloatType()),
    StructField("bikeid", IntegerType()),
    StructField("membership_type", StringType()),
    StructField("usertype", StringType()),
    StructField("birth_year", IntegerType()),
    StructField("gender", IntegerType()),
])
options = {
    "FIELD_OPTIONALLY_ENCLOSED_BY": '"',
    "NULL_IF": '',
}
raw_features = session.read.options(options).schema(schema).csv(f"@{STAGE_NAME}")

# OPTIONAL: Limit to 100K rows and save to a temp table for this exercise.
#           The source data is fairly large so queries may take a while to run
#           if we use the entire dataset.
table_name = f"{DATABASE_NAME}.{SCHEMA_NAME}.citibike_trips_table"
raw_features.limit(100000).write.save_as_table(table_name, mode="overwrite", table_type="temp")
raw_features = session.table(table_name)

# Show a preview of the data using snowpark.DataFrame.to_pandas()
raw_features.limit(5).to_pandas()

,TRIPDURATION,STARTTIME,STOPTIME,START_STATION_ID,START_STATION_NAME,START_STATION_LATITUDE,START_STATION_LONGITUDE,END_STATION_ID,END_STATION_NAME,END_STATION_LATITUDE,END_STATION_LONGITUDE,BIKEID,MEMBERSHIP_TYPE,USERTYPE,BIRTH_YEAR,GENDER
0,745,2015-06-08 07:30:00,2015-06-08 07:43:00,3002,South End Ave & Liberty St,40.711512,-74.015756,127,Barrow St & Hudson St,40.731724,-74.006744,17771,None,Subscriber,1978.0,1
1,3184,2015-06-08 07:31:00,2015-06-08 08:24:00,217,Old Fulton St,40.702772,-73.993836,398,Atlantic Ave & Furman St,40.691652,-73.999979,16156,None,Customer,NaN,0
2,863,2015-06-08 07:31:00,2015-06-08 07:45:00,251,Mott St & Prince St,40.723180,-73.994800,387,Centre St & Chambers St,40.712733,-74.004607,15659,None,Subscriber,1961.0,1
3,318,2015-06-08 07:31:00,2015-06-08 07:36:00,271,Ashland Pl & Hanson Pl,40.685282,-73.978058,310,State St & Smith St,40.689269,-73.989129,21499,None,Subscriber,1980.0,1
4,466,2015-06-08 07:31:00,2015-06-08 07:39:00,285,Broadway & E 14 St,40.734546,-73.990741,472,E 32 St & Park Ave,40.745712,-73.981948,15947,None,Subscriber,1971.0,1


Create a new Feature Store
=========================================

Create a new Feature Store from ``DATABASE_NAME`` and ``SCHEMA_NAME``. Note that we also configure a
``default_warehouse`` to be used with the Feature Store. The choice of warehouse is not important at
this time so long as a valid warehouse is provided.

In [4]:
from snowflake.ml.feature_store import FeatureStore, CreationMode, Entity, FeatureView

fs = FeatureStore(
    session=session,
    database=DATABASE_NAME,
    name=SCHEMA_NAME,
    default_warehouse=WAREHOUSE_NAME,
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST,
)

## Creating Entities

An *entity* is an abstraction over a set of primary keys used for looking up feature data. An Entity represents a real-world "thing" that has data associated with it. Below cell registers an entity called "route" in Feature Store.

In [5]:
entity = Entity(
    name="route",
    join_keys=["START_STATION_ID", "END_STATION_ID"],
    desc="Starting and ending stations for the bike ride"
)
fs.register_entity(entity)

# Show our newly created entity
# snowpark.DataFrame.show() is another way to preview the DataFrame contents
fs.list_entities().show()

--------------------------------------------------------------------------------------------------------------
|"NAME"  |"JOIN_KEYS"                          |"DESC"                                          |"OWNER"     |
--------------------------------------------------------------------------------------------------------------
|ROUTE   |["START_STATION_ID,END_STATION_ID"]  |Starting and ending stations for the bike ride  |REGTEST_RL  |
--------------------------------------------------------------------------------------------------------------



## Creating Feature Views

A *feature view* is a group of logically-related features that are refreshed on the same schedule. The
`FeatureView` constructor accepts a Snowpark DataFrame that contains the feature generation logic. The provided
DataFrame must contain the `join_keys` columns specified in the entities associated with the feature view. In
this example we are using time-series data, so we will also specify the timestamp column name. 

Below cell creates a feature view with 4 features. These 4 features are averaged TRIPDURATION value over past X (1 day, 7 days, 30 days and 1 year) time period and grouped by entity (START_STATION_ID and END_STATION_ID). It uses the [Snowpark analytics function](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/snowpark/api/snowflake.snowpark.DataFrameAnalyticsFunctions.time_series_agg) for time-series aggreation. 

In [6]:
from snowflake.snowpark import functions as F

trip_stats = raw_features.select(
    F.col("STOPTIME"),
    F.col("START_STATION_ID"),
    F.col("END_STATION_ID"),
    F.col("TRIPDURATION"),
).dropna().analytics.time_series_agg(
    time_col="STOPTIME",
    group_by=entity.join_keys,
    aggs={"TRIPDURATION": ["AVG"]},
    windows=["-1D", "-7D", "-30D", "-1Y"],
    sliding_interval="1D",
    col_formatter=lambda input_col, agg, window : f"{input_col}_{agg}_{window.lstrip('-')}",
).drop(F.col("SLIDING_POINT"), F.col("TRIPDURATION"))

trip_stats_fv = FeatureView(
    name="trip_history",
    entities=[entity],
    feature_df=trip_stats,
    timestamp_col="STOPTIME",
)

trip_stats_fv = fs.register_feature_view(trip_stats_fv, version="1", overwrite=True)

# Show our newly created Feature View and display as Pandas DataFrame
fs.list_feature_views().to_pandas()

DataFrameAnalyticsFunctions.time_series_agg() is experimental since 1.12.0. Do not use it in production. 
DataFrame.alias() is experimental since 1.5.0. Do not use it in production. 


,NAME,VERSION,DATABASE_NAME,SCHEMA_NAME,CREATED_ON,OWNER,DESC,ENTITIES,REFRESH_FREQ,REFRESH_MODE,SCHEDULING_STATE,WAREHOUSE
0,TRIP_HISTORY,1,REGTEST_DB,SNOWFLAKE_FEATURE_STORE_NOTEBOOK_QUICK_START,2024-07-22 10:48:05.340,REGTEST_RL,,"[\n ""ROUTE""\n]",None,None,None,None


## Generating Datasets for Training

We are now ready to generate our training set. We'll define a spine DataFrame to form the backbone of our generated
dataset and pass it into ``FeatureStore.generate_dataset()`` along with our Feature Views.

> NOTE: The spine serves as a request template and specifies the entities, labels and timestamps (when applicable). The
  feature store then attaches feature values along the spine using an AS-OF join to efficiently combine and serve
  the relevant, point-in-time correct feature data.

In [7]:
# Build our spine DF, filtering to rides that we have at least 10 records for to ensure statistical significance.
# Note that we use STARTTIME as the timestamp in our spine, which will be matched to the timestamp column(s) in
# the FeatureView. In this case, trip_stats_fv uses STOPTIME as its timestamp column, meaning each record in the
# spine will only be joined to rides that were completed prior to the current record.
query = f"""
    WITH routes AS (
    SELECT START_STATION_ID, END_STATION_ID
    FROM {table_name}
    GROUP BY START_STATION_ID, END_STATION_ID
    HAVING COUNT(*) >= 10
    )
    SELECT t.STARTTIME, t.START_STATION_ID, t.END_STATION_ID, t.TRIPDURATION
    FROM {table_name} t
    JOIN routes r
    ON t.START_STATION_ID = r.START_STATION_ID AND t.END_STATION_ID = r.END_STATION_ID
"""
spine_df = session.sql(query)

ds = fs.generate_dataset(
    name="trip_duration_ds",
    spine_df=spine_df,
    features=[trip_stats_fv],
    spine_timestamp_col="STARTTIME",
    spine_label_cols=["TRIPDURATION"],
    include_feature_view_timestamp_col=False,   # optional
)

# Show preview of the Dataset contents by loading into a Pandas DataFrame
ds.read.to_pandas().head(5)

,STARTTIME,START_STATION_ID,END_STATION_ID,TRIPDURATION,TRIPDURATION_AVG_1D,TRIPDURATION_AVG_7D,TRIPDURATION_AVG_30D,TRIPDURATION_AVG_1Y
0,2013-10-16 18:39:51,453,521,296,NaN,NaN,NaN,NaN
1,2013-12-17 22:17:16,453,521,344,296.0,296.0,296.0,296.000000
2,2014-01-13 17:07:57,453,521,252,344.0,344.0,344.0,320.000000
3,2014-02-26 21:46:39,453,521,239,252.0,252.0,298.0,297.333344
4,2014-05-06 18:48:38,453,521,400,239.0,239.0,239.0,282.750000


In [8]:
# IGNORE: Sanity check output from previous cell
assert len(_) > 0

You can now use this dataset in your downstream modeling workloads. Models trained using Snowpark ML Modeling
and Snowflake Model Registry will automatically benefit from model lineage and other MLOps features.
You can find full examples of using the Snowflake Feature Store on GitHub at
[`snowflake-ml-python`](https://github.com/snowflakedb/snowflake-ml-python/tree/main/snowflake/ml/feature_store/notebooks/customer_demo>)

## Clean Up

To clean up the resources created during this example, just drop the schema we created at the beginning.

In [9]:
session.sql(f"DROP SCHEMA {DATABASE_NAME}.{SCHEMA_NAME}").collect()

[Row(status='SNOWFLAKE_FEATURE_STORE_NOTEBOOK_QUICK_START successfully dropped.')]